In [1]:
import pandas
with open("nba_2013.csv", 'r') as csvfile:
 nba = pandas.read_csv(csvfile)

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
nba.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [4]:
nba.shape

(481, 31)

In [5]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player        481 non-null    object 
 1   pos           481 non-null    object 
 2   age           481 non-null    int64  
 3   bref_team_id  481 non-null    object 
 4   g             481 non-null    int64  
 5   gs            481 non-null    int64  
 6   mp            481 non-null    int64  
 7   fg            481 non-null    int64  
 8   fga           481 non-null    int64  
 9   fg.           479 non-null    float64
 10  x3p           481 non-null    int64  
 11  x3pa          481 non-null    int64  
 12  x3p.          414 non-null    float64
 13  x2p           481 non-null    int64  
 14  x2pa          481 non-null    int64  
 15  x2p.          478 non-null    float64
 16  efg.          479 non-null    float64
 17  ft            481 non-null    int64  
 18  fta           481 non-null    

In [6]:
nba.isnull().sum()

player           0
pos              0
age              0
bref_team_id     0
g                0
gs               0
mp               0
fg               0
fga              0
fg.              2
x3p              0
x3pa             0
x3p.            67
x2p              0
x2pa             0
x2p.             3
efg.             2
ft               0
fta              0
ft.             20
orb              0
drb              0
trb              0
ast              0
stl              0
blk              0
tov              0
pf               0
pts              0
season           0
season_end       0
dtype: int64

In [7]:
nba['fg.'].fillna(nba['fg.'].mean(),inplace=True)

In [8]:
nba['x3p.'].fillna(nba['x3p.'].mean(),inplace=True)

In [9]:
nba['ft.'].fillna(nba['ft.'].mean(),inplace=True)

In [10]:
nba['x2p.'].fillna(nba['x2p.'].mean(),inplace=True)

In [11]:
nba['efg.'].fillna(nba['efg.'].mean(),inplace=True)

In [12]:
nba.isnull().sum()

player          0
pos             0
age             0
bref_team_id    0
g               0
gs              0
mp              0
fg              0
fga             0
fg.             0
x3p             0
x3pa            0
x3p.            0
x2p             0
x2pa            0
x2p.            0
efg.            0
ft              0
fta             0
ft.             0
orb             0
drb             0
trb             0
ast             0
stl             0
blk             0
tov             0
pf              0
pts             0
season          0
season_end      0
dtype: int64

In [13]:
nba.drop(columns=['season','season_end'],axis=1,inplace=True)

In [14]:
nba['pos'].unique()

array(['SF', 'C', 'PF', 'SG', 'PG', 'G', 'F'], dtype=object)

In [15]:
nba['bref_team_id'].unique()

array(['TOT', 'OKC', 'ORL', 'NOP', 'NYK', 'POR', 'MIA', 'MEM', 'BRK',
       'PHI', 'MIL', 'ATL', 'WAS', 'GSW', 'DEN', 'HOU', 'SAS', 'BOS',
       'PHO', 'MIN', 'LAC', 'CLE', 'UTA', 'DET', 'CHA', 'DAL', 'CHI',
       'LAL', 'IND', 'TOR', 'SAC'], dtype=object)

In [16]:
nba['player']=nba['player'].factorize()[0]

In [17]:
bref_team_id_map=nba['bref_team_id'].value_counts().to_dict()

In [18]:
nba['bref_team_id']=nba['bref_team_id'].map(bref_team_id_map)

In [19]:
pos_map={
    'SF':1,
    'C':2,
    'PF':3,
    'SG':4,
    'PG':5,
    'G':6,
    'F':7
}

In [20]:
nba['pos']=nba['pos'].map(pos_map)

In [21]:
nba.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,0,1,23,63,63,0,847,66,141,0.468,...,0.660,72,144,216,28,23,26,30,122,171
1,1,2,20,17,81,20,1197,93,185,0.503,...,0.581,142,190,332,43,40,57,71,203,265
2,2,3,27,63,53,12,961,143,275,0.520,...,0.639,102,204,306,38,24,36,39,108,362
3,3,4,28,13,73,73,2552,464,1011,0.459,...,0.815,32,230,262,248,35,3,146,136,1330
4,4,2,25,18,56,30,951,136,249,0.546,...,0.836,94,183,277,40,23,46,63,187,328


In [22]:
y=nba['pts']
x=nba.drop('pts',axis=1)

In [23]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.25)

In [25]:
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [26]:
y_pred = knn.predict(x_test)
y_pred

array([ 109.2,   49.4,  414.6,   64.6,  162. , 1075.4,    4.6,  412.4,
        103.4,  657.6,  414.4, 1582.2,    9.8,  181.2,   35.2,  157. ,
        115.8, 1417.2,  185.4, 1433.4,    2.8,  736. ,   91.2,  202. ,
       1143.8,   31.8,    4. ,    8.8,    9.8,  914.4, 1317.6, 1382.6,
        370.6,  190.6,  887.8,  853. ,  100.6,  952.4,  298.4,  865.2,
        225.4,  315.6,    5.6,  365.4,  585.2,  612. ,   24.6,  635.4,
        792. ,    9.2,  800.8,  515.6,  302.4,  149.8,  772.2, 1696.6,
         43.2,  238.8, 1152.6,  693.4,   46.8,  162.2, 1787. ,  950. ,
         12.6,  966.4, 1173.8,  861.8,   12.2,  618.8,  416.2,  514.4,
        779.8,  194.4,  273.8,  103.6,  300.4,  866.8,   92.4,  136.8,
        187. ,  308.2,  388.4, 1058.6,  290.4, 1334.6,  292.8,  193.4,
         77.8,  932.2,  234.2,  170. ,  287.4, 1036.2,   86. ,  638.8,
        269.6,    5.8,  202.2, 1376.4,  223.4,  678.2,   67.8,   20.4,
        546.8,   86. ,  615.2,  103.4,   12.2, 1207.2,  289. ,   80.4,
      

In [27]:
knn.score(x_train,y_train)

0.9815829285981785

In [28]:
knn.score(x_test,y_test)

0.9756412352992921